# [Spotahome.com](https://www.spotahome.com/s/london--uk/for-rent:apartments/for-rent:studios/bedrooms:1/bedrooms:2/bedrooms:3/bedrooms:3more?areaId[]=219&areaId[]=231&areaId[]=232&areaId[]=233&areaId[]=234&areaId[]=235&areaId[]=236&areaId[]=237&areaId[]=241)

## Dataframe with information from search result page

In [1]:
import requests                 # requests on websites
from bs4 import BeautifulSoup   # html parsing
import pandas as pd             # pandas for data frame
import math                     # for math methods
import time                     # for sleep timer
import random
from sql_functions import *     # functions from file for upload on schema
import psycopg2                 # for upload on engine
import datetime as dt           # for the csv file with the current date and time
import re


def get_description(bs):
    lst_name = []
    descriptions = bs.find_all(
        class_='homecard-content__title__HomecardContent___OmV4c homecard-content__title--rebranding-style__HomecardContent___OmV4c')
    for description in descriptions:
        lst_name.append(
            description.get_text()
                .strip()
        )
    return lst_name


def get_housing(bs):
    lst_name = []
    housings = bs.find_all(
        class_='homecard-content__type__HomecardContent___OmV4c homecard-content__type--rebranding-style__HomecardContent___OmV4c')
    for housing in housings:
        lst_name.append(
            housing.get_text()
                .strip()
        )
    return lst_name


def get_available(bs):
    lst_name = []
    availables = bs.find_all(
        class_='homecard-content__available-from__HomecardContent___OmV4c homecard-content__available-from--rebranding-style__HomecardContent___OmV4c')
    for available in availables:
        lst_name.append(
            available.get_text()
                .strip()
                .replace('From ', '')
        )
    return lst_name


def get_price(bs):
    lst_name = []
    prices = bs.find_all(class_='price__Price___OmV4c')
    for price in prices:
        lst_name.append(
            price.get_text()
                .strip()
                .replace('£', '')
        )
    return lst_name


def get_prices_period(bs):
    lst_name = []
    prices_period = bs.find_all(
        class_='price-monthly__Price___OmV4c price-monthly--rebranding-style__Price___OmV4c')
    for price_period in prices_period:
        lst_name.append(
            price_period.get_text()
                .strip()
                .replace('/', '')
        )
    return lst_name


def get_ids(bs):
    lst_name = []
    ids = bs.find_all(class_='l-list__item')
    for id in ids:
        lst_name.append(
            id.get('data-homecard-scroll')
                .strip()
        )
    return lst_name


# Create dictionary in which every location ID gets assigned a location name
location_dict = {219: 'Lambeth',
                 231: 'Hammersmith and Fulham',
                 232: 'Kensington and Chelsea',
                 233: 'City of Westminster',
                 234: 'Camden',
                 235: 'Tower Hamlets',
                 236: 'Islington',
                 237: 'Hackney',
                 241: 'City of London'
                 }


# the website spotahome shows 60 search results per page. To iterate trough all the pages, we get the information how many search results are there, then divide it by 60 and round it up to get the number of pages.
def page_results(property_type, location):
    page = requests.get(
        f'https://www.spotahome.com/s/london--uk/for-rent:{property_type}?areaId[]={location}')
    html = page.content
    bs = BeautifulSoup(html, 'html.parser')

    # Extracting the total number of search results
    results = bs.find_all('h1', {'class': 'search-title__title'})

    result_text = 0

    for result in results:
        result_text = result.find("strong").get_text().strip()

    # convert the extracted string to an integer to perform mathematical operations
    result_converted = int(result_text)

    # divide the converted result by 60 since one pages shows 60 results and round it up to get the number of pages
    page_site = result_converted / 60
    page_site = math.ceil(page_site)

    # convert the number of pages from a float to an integer to iterate through the pages
    page_converted = int(page_site)

    df_search = pd.DataFrame()

    begin = f'https://www.spotahome.com/s/london--uk/for-rent:{property_type}'
    end = f'?areaId[]={location}'

    # range is including in the beginning and excluding in the end so we add plus 1 to iterate through all calculated pages
    page_converted = page_converted + 1

    for page_number in range(page_converted):
        time.sleep(random.randint(2,6)/10)
        page = requests.get(begin+f'/page:{page_number}'+end)
        html = page.content
        bs = BeautifulSoup(html, 'html.parser')

        # Create a dictionary to store the results from every loop cycle.
        # The keys are the column names and the values are the functions we created before.
        # The functions are called with the beautiful soup object as a parameter.

        spotahome_dict = {
            'platform_id': get_ids(bs),
            'platform': 'Spotahome',
            'neighborhood': location_dict[location],
            'housing_type': property_type,
            'property_type': get_housing(bs),
            'price_pcm': get_price(bs),
            'title': get_description(bs),
            'furnished': 'furnished',
            'available_from': get_available(bs),
            'let_type': '',
            'available_today': '',
            'scraping_date': dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        }
        df_page = pd.DataFrame(data=spotahome_dict)

        # the temporary data frame stores the data to the data frame we created earlier outside the for loop
        # for every iteration, the data frame page stores the results in the data frame search
        df_search = pd.concat([df_search, df_page], axis=0, ignore_index=True)
    return (df_search)


property_types = ['studios', 'apartments/bedrooms:1',
                  'apartments/bedrooms:2', 'apartments/bedrooms:3', 'apartments/bedrooms:3more']
locations = [219, 231, 232, 233, 234, 235, 236, 237, 241]

df_complete = pd.DataFrame()
for property_type in property_types:
    for location in locations:
        df_complete = pd.concat([df_complete, page_results(
            property_type, location)], axis=0, ignore_index=True)

# split the date in seperate columns for further analysis
df_complete = df_complete.join(df_complete['price_pcm'].str.split('-', expand=True).add_prefix('price_pcm_'))
df_complete = df_complete.join(df_complete['available_from'].str.split(' ', expand=True).add_prefix('available_from_'))


In [2]:
display(df_complete)
df_complete.info()

,platform_id,platform,neighborhood,housing_type,property_type,price_pcm,title,furnished,available_from,let_type,available_today,scraping_date,price_pcm_0,price_pcm_1,available_from_0,available_from_1,available_from_2
0,654298,Spotahome,Hammersmith and Fulham,studios,Studio,2650,The Armadale Road Residence II,furnished,16 December,,,2022-10-05 14:23:09,2650,None,16,December,None
1,654299,Spotahome,Hammersmith and Fulham,studios,Studio,2500,The Armadale Road Residence III,furnished,20 December,,,2022-10-05 14:23:09,2500,None,20,December,None
2,113567,Spotahome,Hammersmith and Fulham,studios,Studio,1400,Studio flat with double bed to rent in Kensing...,furnished,18 June 2023,,,2022-10-05 14:23:09,1400,None,18,June,2023
3,761515,Spotahome,Kensington and Chelsea,studios,Studio,2250,Studio for rent in a co-living building in Not...,furnished,10 October,,,2022-10-05 14:23:12,2250,None,10,October,None
4,607194,Spotahome,Kensington and Chelsea,studios,Studio,3033,"Studio for rent in Earls Court, London",furnished,12 October,,,2022-10-05 14:23:12,3033,None,12,October,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829,844665,Spotahome,Hackney,apartments/bedrooms:3more,Apartment,8250-10950,4-bedroom apartment for rent in London,furnished,8 January 2023,,,2022-10-05 14:25:12,8250,10950,8,January,2023
830,843245,Spotahome,Hackney,apartments/bedrooms:3more,Apartment,9000-11250,4-bedroom apartment for rent in London,furnished,11 January 2023,,,2022-10-05 14:25:12,9000,11250,11,January,2023
831,398966,Spotahome,Hackney,apartments/bedrooms:3more,Apartment,2100,4-bedroom apartment in Hackney,furnished,6 January 2024,,,2022-10-05 14:25:12,2100,None,6,January,2024
832,602243,Spotahome,Hackney,apartments/bedrooms:3more,Apartment,9000-11250,4 bedrooms warehouse conversion for rent in Ha...,furnished,4 August 2023,,,2022-10-05 14:25:12,9000,11250,4,August,2023


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 834 entries, 0 to 833
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   platform_id       834 non-null    object
 1   platform          834 non-null    object
 2   neighborhood      834 non-null    object
 3   housing_type      834 non-null    object
 4   property_type     834 non-null    object
 5   price_pcm         834 non-null    object
 6   title             834 non-null    object
 7   furnished         834 non-null    object
 8   available_from    834 non-null    object
 9   let_type          834 non-null    object
 10  available_today   834 non-null    object
 11  scraping_date     834 non-null    object
 12  price_pcm_0       834 non-null    object
 13  price_pcm_1       245 non-null    object
 14  available_from_0  834 non-null    object
 15  available_from_1  834 non-null    object
 16  available_from_2  434 non-null    object
dtypes: object(17)
me

In [3]:
df_complete.head(5)

,platform_id,platform,neighborhood,housing_type,property_type,price_pcm,title,furnished,available_from,let_type,available_today,scraping_date,price_pcm_0,price_pcm_1,available_from_0,available_from_1,available_from_2
0,654298,Spotahome,Hammersmith and Fulham,studios,Studio,2650,The Armadale Road Residence II,furnished,16 December,,,2022-10-05 14:23:09,2650,None,16,December,None
1,654299,Spotahome,Hammersmith and Fulham,studios,Studio,2500,The Armadale Road Residence III,furnished,20 December,,,2022-10-05 14:23:09,2500,None,20,December,None
2,113567,Spotahome,Hammersmith and Fulham,studios,Studio,1400,Studio flat with double bed to rent in Kensing...,furnished,18 June 2023,,,2022-10-05 14:23:09,1400,None,18,June,2023
3,761515,Spotahome,Kensington and Chelsea,studios,Studio,2250,Studio for rent in a co-living building in Not...,furnished,10 October,,,2022-10-05 14:23:12,2250,None,10,October,None
4,607194,Spotahome,Kensington and Chelsea,studios,Studio,3033,"Studio for rent in Earls Court, London",furnished,12 October,,,2022-10-05 14:23:12,3033,None,12,October,None


## Dataframe with information from every detail page for every apartment advert

In [4]:
df_details_complete = pd.DataFrame()

#with iterrows we can grab the id's from the previous code, iterate trough all of them and get the details for every id
for idx, row in df_complete.iterrows():
    time.sleep(random.randint(2,6)/10)
    page = f"https://www.spotahome.com/london/for-rent:{row['housing_type'].lower() + 's'}/{row['platform_id']}"
    website = requests.get(page)
    results = BeautifulSoup(website.content, 'html.parser')

    details = results.find(
        'div', class_='property-title__details').find_all('span')

    #our information is first stored in a list
    details_lst = (detail.get_text() for detail in details)
    details_lst = [detail.strip() for detail in details_lst]

    #since we don't need the property_type again, we drop that information
    details_lst.pop(0)

    #we can split our information on the space and get three lists inside a list
    details_lst = [i.split(' ', 1) for i in details_lst]

    row_dict = {}

    #the three lists are now get stored in a dictionary (with the id's)
    for value_key_tuple in details_lst:
        new_key_value = {'id': row['platform_id']}
        row_dict.update(new_key_value)
        key = value_key_tuple[1]
        value = value_key_tuple[0]
        row_dict[key] = value

    #the dictionary is then converted to a data frame
    df_details = pd.DataFrame(data=row_dict, index=[0])
    df_details_complete = pd.concat([df_details_complete, df_details])


AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
display(df_details_complete)
df_details_complete.info()

## Import df_complete to DBeaver

In [ ]:
# call the schema created for this project
schema = 'capstone_jmrs'
# get the function to connect to the database
engine = get_engine()

# give the table a unique name
today = dt.datetime.today().strftime('%Y-%m-%d-%H-%M')
table_name = f"spotahome_df_complete_{today}"

# import the table to sql
if engine != None:
    try:
        df_complete.to_sql(name=table_name,
                           con=engine,
                           if_exists='replace',
                           schema=schema,
                           index=False,
                           chunksize=5000,
                           method='multi')
        print(f"The {table_name} table was imported successfully.")

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

## Import df_details_complete to DBeaver

In [ ]:
# call the schema created for this project
schema = 'capstone_jmrs'
# get the function to connect to the database
engine = get_engine()

# give the table a unique name
today = dt.datetime.today().strftime('%Y-%m-%d-%H-%M')
table_name = f"spotahome_df_details_complete_{today}"

# import the table to sql
if engine != None:
    try:
        df_details_complete.to_sql(name=table_name,
                                   con=engine,
                                   if_exists='replace',
                                   schema=schema,
                                   index=False,
                                   chunksize=5000,
                                   method='multi')
        print(f"The {table_name} table was imported successfully.")

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None


## Create a .csv file for both data frames

In [ ]:
# create a .csv file with the current date and time
today = dt.datetime.today().strftime('%Y-%m-%d-%H-%M')
df_complete.to_csv('spotahome_df_complete_{}.csv'.format(today), sep='\t')


today = dt.datetime.today().strftime('%Y-%m-%d-%H-%M')
df_details_complete.to_csv('spotahome_df_details_complete{}.csv'.format(today), sep='\t')